In [1]:

#название источника,
#наименование новости,
#ссылку на новость,
#дата публикации

from pprint import pprint
from lxml import html
import requests
import datetime


#----------------------------------------------
#Яндекс

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

response = requests.get('https://yandex.ru/news',
                            headers=header)
dom = html.fromstring(response.text)
my_news= []
items = dom.xpath("(//table[@class='stories-set__items'])[3]//td[@class='stories-set__item']")
for item in items:
    new= {}
    name = item.xpath("(.//a[contains(@class, 'link link_theme_black')])[1]/text()")[0]
    source_name= item.xpath("(.//div[contains(@class, 'story__date')])[1]/text()")[0][0: -6]
    source_date =  item.xpath("(.//div[contains(@class, 'story__date')])[1]/text()")[0][-5:]
    d = datetime.date.today()

    source_date = datetime.datetime(d.year, d.month, d.day, int(source_date[0:1]), int(source_date[3:4]), 0)
    link = item.xpath("(.//a[contains(@class, 'link link_theme_black')])[1]/@href")[0] 

    new['name'] = name
    new['source_name'] = source_name
    new['source_date'] = source_date
    new['link'] = link

    my_news.append(new)

#pprint(my_news)
   





In [2]:
#Лента

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
main_link = 'https://lenta.ru'
response = requests.get(main_link,
                            headers=header)
dom = html.fromstring(response.text)

lenta_news= []
items = dom.xpath("(//section[@class='row b-top7-for-main js-top-seven'])[1]//div[@class='item']|//div[@class='first-item']")
#print(items)
for item in items:
    new= {}
    name = item.xpath(".//a[1]/text()")[0] 
    name = name.replace(u'\xa0', u' ')
    source_name= 'lenta.ru' 
    source_date = item.xpath(".//a[1]/time[1]/text()")[0]
    d = datetime.date.today()

    source_date = datetime.datetime(d.year, d.month, d.day, int(source_date[0:1]), int(source_date[3:4]), 0)
    link = main_link + item.xpath(".//a[1]/@href")[0] 

    new['name'] = name
    new['source_name'] = source_name
    new['source_date'] = source_date
    new['link'] = link

    my_news.append(new)

#pprint(my_news)



   

In [3]:
#Маил
#from datetime import datetime

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
main_link = 'https://news.mail.ru/?from=menu'
response = requests.get(main_link,
                            headers=header)
dom = html.fromstring(response.text)

mail_news= []
items1 = dom.xpath("(//div[@class='cols__inner'])[5]")[0]
items = items1.xpath(".//li[@class='list__item']|.//div[@class='newsitem newsitem_height_fixed js-ago-wrapper']")

#items = items1.xpath("//div[@class='cols__inner'][5]//div[@class='newsitem newsitem_height_fixed js-ago-wrapper']|/li[@class='list__item']")
#//div[@class='newsitem newsitem_height_fixed js-ago-wrapper']
# //div[@class='newsitem newsitem_height_fixed js-ago-wrapper']

for item in items:
    new= {}
    name = item.xpath(".//a[1]/span/text()")[0] 
    link = item.xpath(".//a[1]/@href")[0] 
    source_name= 'lenta.ru'
    #запрос к странице новости:
    response_mail = requests.get(link,
                            headers=header)
    dom_mail = html.fromstring(response_mail.text)
    d = dom_mail.xpath("(//div[@class='breadcrumbs breadcrumbs_article js-ago-wrapper'])[1]//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    source_name = dom_mail.xpath("//a[contains(@class, 'breadcrumbs__link')]/span/text()")[0]
    source_date = datetime.datetime(int(d[0:4]), int(d[5:7]), int(d[8:10]), int(d[11:13]),int(d[14:16]), int(d[17:19]))

    

    new['name'] = name
    new['source_name'] = source_name
    new['source_date'] = source_date
    new['link'] = link
    my_news.append(new)

pprint(my_news)

[{'link': '/news/story/Krylya_Sovetov_obygrali_Arsenal_v_chempionate_Rossii_po_futbolu--d693aa874f685ac4dc94a23f9123f713?lr=22&lang=ru&stid=Vb341NNIlXa4whFo8l44&persistent_id=104884252&rubric=personal_feed&from=index',
  'name': '«Крылья Советов» обыграли «Арсенал» в чемпионате России по футболу',
  'source_date': datetime.datetime(2020, 7, 7, 2, 3),
  'source_name': 'РИА Новости'},
 {'link': '/news/story/Dzhonni_Depp_i_EHmber_KHyord_vstretilis_v_sude--e6586892eda7dfd236dc42d41f0248e2?lr=22&lang=ru&stid=xYf1dfvPt5BqWnmolt-2&persistent_id=104243627&rubric=personal_feed&from=index',
  'name': 'Джонни Депп и Эмбер Хёрд встретились в суде',
  'source_date': datetime.datetime(2020, 7, 7, 2, 1),
  'source_name': 'НТВ'},
 {'link': '/news/story/Zoloto_obnovilo_cenovoj_maksimum_s_2011_goda--f16a1c4566d315aabae5faf23dd78f75?lr=22&lang=ru&stid=6yE8SwbpyP33de0_Y6aM&persistent_id=104931192&rubric=personal_feed&from=index',
  'name': 'Золото обновило ценовой максимум с 2011 года',
  'source_date': d

In [4]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['vacancy_db']

news = db.news
news.insert_many(my_news)